In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [3]:
phys_net = PhysNet(norm='layer')
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')
phys_net.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:54630323
Model: "phys_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 128, 32, 32, 16)   1472      
_________________________________________________________________
sequential_1 (Sequential)    (None, 128, 16, 16, 32)   14112     
_________________________________________________________________
sequential_2 (Sequential)    (None, 128, 16, 16, 64)   55616     
_________________________________________________________________
sequential_3 (Sequential)    (None, 64, 8, 8, 64)      110784    
_________________________________________________________________
sequential_4 (Sequential)    (None, 64, 8, 8, 64)      110784    
_________________________________________________________________
sequential_5 (Sequential)    (None, 32, 4, 4, 64)      110720    
__________

In [4]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet-LN_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet-LN_CCNU.h5')

Epoch 1/3
2329/2329 [==============================] - 1306s 554ms/step - loss: 0.3017 - val_loss: 0.2744

Epoch 00001: val_loss improved from inf to 0.27441, saving model to ../weights\PhysNet-LN_CCNU.h5
Epoch 2/3
2329/2329 [==============================] - 1295s 553ms/step - loss: 0.2304 - val_loss: 0.2876

Epoch 00002: val_loss did not improve from 0.27441
Epoch 3/3
2329/2329 [==============================] - 1294s 553ms/step - loss: 0.2122 - val_loss: 0.2511

Epoch 00003: val_loss improved from 0.27441 to 0.25112, saving model to ../weights\PhysNet-LN_CCNU.h5


In [4]:
phys_net.load_weights('../weights/PhysNet-LN_CCNU.h5')

In [5]:
eval_on_dataset(test_set_CCNU, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_CCNU_CCNU.h5')

100%|██████████| 179/179 [02:46<00:00,  1.08it/s]


{'Sliding window': {'MAE': 1.279, 'RMSE': 5.287, 'R': 0.86218},
 'Whole video': {'MAE': 0.626, 'RMSE': 2.721, 'R': 0.95782}}

In [6]:
eval_on_dataset(test_set_CCNU_rPPG, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_CCNU_CCNU_rPPG.h5')

100%|██████████| 54/54 [00:33<00:00,  1.60it/s]


{'Sliding window': {'MAE': 0.88, 'RMSE': 3.344, 'R': 0.93993},
 'Whole video': {'MAE': 0.565, 'RMSE': 1.728, 'R': 0.98321}}

In [7]:
eval_on_dataset(test_set_PURE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_CCNU_PURE.h5')

100%|██████████| 59/59 [00:26<00:00,  2.20it/s]


{'Sliding window': {'MAE': 0.756, 'RMSE': 4.733, 'R': 0.97883},
 'Whole video': {'MAE': 0.385, 'RMSE': 0.589, 'R': 0.99976}}

In [8]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:15<00:00,  2.71it/s]


{'Sliding window': {'MAE': 1.566, 'RMSE': 4.016, 'R': 0.97316},
 'Whole video': {'MAE': 1.006, 'RMSE': 1.491, 'R': 0.99685}}